In [1]:
from json import dumps
from time import sleep
from kafka import KafkaProducer
from kafka import KafkaConsumer
from json import loads

In [4]:
consumer = KafkaConsumer(
    'my-topic',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))

my-topic:0:0: key=None value={'number': 0}
my-topic:0:1: key=None value={'number': 1}
my-topic:0:2: key=None value={'number': 2}
my-topic:0:3: key=None value={'number': 3}
my-topic:0:4: key=None value={'number': 4}
my-topic:0:5: key=None value={'number': 5}
my-topic:0:6: key=None value={'number': 6}
my-topic:0:7: key=None value={'number': 7}
my-topic:0:8: key=None value={'number': 8}
my-topic:0:9: key=None value={'number': 9}
my-topic:0:10: key=None value={'number': 10}
my-topic:0:11: key=None value={'number': 11}
my-topic:0:12: key=None value={'number': 0}
my-topic:0:13: key=None value={'number': 1}
my-topic:0:14: key=None value={'number': 2}
my-topic:0:15: key=None value={'number': 3}
my-topic:0:16: key=None value={'number': 4}
my-topic:0:17: key=None value={'number': 5}
my-topic:0:18: key=None value={'number': 6}
my-topic:0:19: key=None value={'number': 7}
my-topic:0:20: key=None value={'number': 8}
my-topic:0:21: key=None value={'number': 9}
my-topic:0:22: key=None value={'number':

my-topic:0:1044: key=None value={'number': 32}
my-topic:0:1045: key=None value={'number': 33}
my-topic:0:1046: key=None value={'number': 34}
my-topic:0:1047: key=None value={'number': 35}
my-topic:0:1048: key=None value={'number': 36}
my-topic:0:1049: key=None value={'number': 37}
my-topic:0:1050: key=None value={'number': 38}
my-topic:0:1051: key=None value={'number': 39}
my-topic:0:1052: key=None value={'number': 40}
my-topic:0:1053: key=None value={'number': 41}
my-topic:0:1054: key=None value={'number': 42}
my-topic:0:1055: key=None value={'number': 43}
my-topic:0:1056: key=None value={'number': 44}
my-topic:0:1057: key=None value={'number': 45}
my-topic:0:1058: key=None value={'number': 46}
my-topic:0:1059: key=None value={'number': 47}
my-topic:0:1060: key=None value={'number': 48}
my-topic:0:1061: key=None value={'number': 49}
my-topic:0:1062: key=None value={'number': 50}
my-topic:0:1063: key=None value={'number': 51}
my-topic:0:1064: key=None value={'number': 52}
my-topic:0:10

my-topic:0:1217: key=None value={'number': 205}
my-topic:0:1218: key=None value={'number': 206}
my-topic:0:1219: key=None value={'number': 207}
my-topic:0:1220: key=None value={'number': 208}
my-topic:0:1221: key=None value={'number': 209}
